In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyomo.environ import *
from pyomo.network import *
from idaes.core.util.model_statistics import degrees_of_freedom
from Method import *

In [2]:
from hda_flowsheet import hda_flowsheet

In [3]:
m = hda_flowsheet()

Building flowsheet...

Setting inputs...

Initializing flowsheet...

Initialization complete for 'R101'! 
Initialization complete for 'F101'! 
model.name="F102";
    - termination condition: infeasible
    - message from solver: Ipopt 3.13.2\x3a Converged to a locally infeasible
      point. Problem may be infeasible.
Initialization complete for 'F102'! 
Initialization complete for 'S101'! 
Initialization complete for 'M101'! 
Initialization complete for 'H101'! 
Initialization complete for 'R101'! 
Initialization complete for 'F101'! 
Initialization complete for 'S101'! 
Initialization complete for 'M101'! 
Initialization complete for 'H101'! 
Initialization complete for 'R101'! 
Initialization complete for 'F101'! 
Initialization complete for 'S101'! 
Initialization complete for 'M101'! 
Initialization complete for 'H101'! 
Initialization complete for 'R101'! 
Initialization complete for 'F101'! 
Initialization complete for 'S101'! 
Initialization complete for 'M101'! 
Initialization

In [4]:
# objectiv, relaxation, constraints

m.objective = Objective(expr=m.operating_cost)

m.H101.Outlet.T.unfix()
m.R101.Q.unfix()
m.F101.Vap_Outlet.T.unfix()
m.F102.Vap_Outlet.T.unfix()
m.F102.Vap_Outlet.P.unfix()

m.H101.Outlet.T.setlb(500)
m.H101.Outlet.T.setub(600)

m.R101.Outlet.T.setlb(600)
m.R101.Outlet.T.setub(800)

m.F101.Vap_Outlet.T.setlb(298.0)
m.F101.Vap_Outlet.T.setub(450.0)
m.F102.Vap_Outlet.T.setlb(298.0)
m.F102.Vap_Outlet.T.setub(450.0)
m.F102.Vap_Outlet.P.setlb(115000)
m.F102.Vap_Outlet.P.setub(165000)

m.overhead_loss = Constraint(expr=m.loss<=0.05)
m.purity_quality = Constraint(expr=m.purity >= 0.80)
m.product_flow = Constraint(expr=m.product >= 0.15)

In [5]:
solver = SolverFactory('ipopt')
solver.options['tol'] = 1e-4
results = solver.solve(m)

In [6]:
print_outcome(m)

Optimal Values: 
	 H101 outlet temperature =  500.0 K
	 R101 outlet temperature =  697.486695484181 K
	 F101 outlet temperature =  308.3442636224523 K
	 F102 outlet temperature =  365.5876719613391 K
	 F102 outlet pressure =  115000.0 Pa

Optimal Outcome: 
	 operating cost  = $ 284116.3035121963
	 benzene purity  = 0.8087768020121621
	 benzene product = 0.14999999003701064
	 Overhead loss   = 0.05000000991257592


In [7]:
stream_information(m)

,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11
温度/˚C,30.0,30.0,32.44,226.85,424.34,35.19,35.19,35.19,35.19,92.44,92.44
压力/kPa,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,115.0,115.0
相态,气相,气相,气液共存,气相,气相,气相,气相,气相,液相,气相,液相
摩尔流量kmol/hr,1.152,1.080,7.036,7.036,7.036,6.004,4.804,1.201,1.031,0.668,0.363
质量流率kg/hr,-,-,-,-,-,-,-,-,-,-,-
->苯,0.0,0.0,16.13,16.13,80.63,20.16,16.13,4.03,60.47,42.18,18.29
->甲苯,0.0,99.51,101.44,101.44,25.36,2.42,1.93,0.48,22.95,11.03,11.92
->甲烷,1.16,0.0,58.28,58.28,71.53,71.41,57.13,14.28,0.12,0.12,0.0
->氢气,2.18,0.0,4.22,4.22,2.56,2.56,2.05,0.51,0.0,0.0,0.0
质量分率,-,-,-,-,-,-,-,-,-,-,-


In [9]:
unit_report(m.s4)

s4: 
Temperature[˚C]:  226.85
Pressure[kPa]:  350.0
Phase type: Vap
flow rate[kmol/hr]: 
	 [ Vap , benzene ]:  0.206
	 [ Vap , toluene ]:  1.101
	 [ Vap , methane ]:  3.633
	 [ Vap , hydrogen ]:  2.095
	 [ Liq , benzene ]:  0.0
	 [ Liq , toluene ]:  0.0
	 [ Liq , methane ]:  0.0
	 [ Liq , hydrogen ]:  0.0
Mass flowrate[kg/hr]:
	 [ benzene ]: 16.125
	 [ toluene ]: 101.444
	 [ methane ]: 58.281
	 [ hydrogen ]: 4.224
Mass fraction[-]:
	 [ benzene ]: 0.089547
	 [ toluene ]: 0.563344
	 [ methane ]: 0.323651
	 [ hydrogen ]: 0.023457
****************************************
